# Task 1: Data Exploration and Enrichment

## Ethiopia Financial Inclusion Forecasting System

**Objective:** Understand the starter dataset and enrich it with additional data useful for forecasting Access (Account Ownership) and Usage (Digital Payment Adoption).

### Outline
1. Load and understand the unified schema
2. Explore the data by record type, pillar, and source
3. Identify temporal range and coverage gaps
4. Review existing events and impact links
5. Document data quality and limitations

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
print('Libraries loaded!')

## 1. Load and Understand the Unified Schema

In [ ]:
# Load the unified dataset
df = pd.read_csv('../data/raw/ethiopia_fi_unified_data.csv')
date_cols = ['observation_date', 'event_date', 'collection_date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f'Dataset loaded: {len(df)} records')
print(f'Columns: {len(df.columns)}')

In [ ]:
# Load reference codes
reference_codes = pd.read_csv('../data/raw/reference_codes.csv')
print('Reference Codes:')
for field in reference_codes['field'].unique():
    codes = reference_codes[reference_codes['field'] == field]['code'].tolist()
    print(f'{field}: {codes}')

## 2. Explore Data Distribution

In [ ]:
# Count records by record_type
print('Records by Type:')
print(df['record_type'].value_counts())

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
record_counts = df['record_type'].value_counts()
colors = ['#2E86AB', '#28A745', '#FFC107', '#DC3545']
axes[0].pie(record_counts.values, labels=record_counts.index, autopct='%1.1f%%', colors=colors[:len(record_counts)])
axes[0].set_title('Distribution by Record Type')

pillar_counts = df[df['pillar'].notna()]['pillar'].value_counts()
axes[1].bar(pillar_counts.index, pillar_counts.values, color=['#2E86AB', '#28A745', '#6C757D'])
axes[1].set_title('Records by Pillar')

obs = df[df['record_type'] == 'observation']
source_counts = obs['source_type'].value_counts()
axes[2].bar(source_counts.index, source_counts.values)
axes[2].set_title('Observations by Source Type')
plt.tight_layout()
plt.savefig('../reports/figures/data_distribution.png', dpi=150)
plt.show()

## 3. Temporal Coverage Analysis

In [ ]:
observations = df[df['record_type'] == 'observation'].copy()
print(f"Temporal Range: {observations['observation_date'].min()} to {observations['observation_date'].max()}")
observations['year'] = observations['observation_date'].dt.year
print(f'Years with data: {sorted(observations["year"].dropna().unique())}')

In [ ]:
# Temporal coverage heatmap
coverage = observations.pivot_table(index='indicator_code', columns='year', values='value_numeric', aggfunc='count').fillna(0)
plt.figure(figsize=(14, 8))
sns.heatmap(coverage, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('Temporal Coverage')
plt.savefig('../reports/figures/temporal_coverage.png', dpi=150)
plt.show()

## 4. Events and Impact Links

In [ ]:
events = df[df['record_type'] == 'event'].sort_values('event_date')
print(f'Total Events: {len(events)}')
for _, e in events.iterrows():
    print(f"{e['event_date'].strftime('%Y-%m-%d') if pd.notna(e['event_date']) else 'N/A'} | {e['category']} | {e['indicator']}")